In [2]:
import openmm
from openmm import app, unit
import pickle

In [3]:
pdb = app.PDBFile("/data/chodera/zhangi/perses_benchmark/neq/14/177/apo_new.pdb")

In [4]:
with open("/data/chodera/zhangi/perses_benchmark/neq/14/177/177_apo.pickle", "rb") as f:
    htf = pickle.load(f)

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [5]:
system = htf._topology_proposal.new_system
positions = pdb.positions

In [6]:
temperature = 300 * unit.kelvin
collision_rate = 1/unit.picosecond
timestep = 4* unit.femtosecond
integrator = openmm.LangevinMiddleIntegrator(temperature, collision_rate, timestep)
    
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

openmm.LocalEnergyMinimizer.minimize(context)

In [7]:
state = context.getState(getPositions=True)

In [8]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, state.getPositions(asNumpy=True), open("/data/chodera/zhangi/perses_benchmark/neq/14/177/apo_new_minimized.pdb", "w"), keepIds=True)